In [2]:
# ==========================================
# Thixo-Metric v1.0 (Final Edition)
# Includes: Granular Workflow & Quantitative Reporting
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from google.colab import files
import io
import os

# PDF Generation Library
!pip install fpdf -q
from fpdf import FPDF

# Set visual style
sns.set_theme(style='whitegrid')
plt.rcParams['font.family'] = 'sans-serif'

# ==========================================
# 1. Data Handling & Evaluation Setup
# ==========================================

# Global trackers
current_data_source = "Synthetic Demo Data"
current_plot = None
df_soil = None
analysis_run = False
pdf_bytes = None # Store generated PDF content here

def create_synthetic_data():
    """Backup synthetic data."""
    np.random.seed(42)
    data = {
        'Sample_ID': [f'BH-{i:03d}' for i in range(1, 11)],
        'Latitude': np.random.uniform(23.5, 24.0, 10),
        'Longitude': np.random.uniform(90.0, 90.5, 10),
        'Depth_m': np.random.uniform(5.0, 20.0, 10),
        'Undisturbed_Su': np.random.uniform(40, 90, 10),
        'Remolded_Su_S0': np.random.uniform(10, 25, 10),
        'PI': np.random.uniform(30, 60, 10),
        'Water_Content': np.random.uniform(35, 70, 10),
        'Liquid_Limit_LL': np.random.uniform(40, 70, 10),
        'is_submerged': np.random.choice([True, False], 10)
    }
    df = pd.DataFrame(data)
    df['Soil_Type'] = df['PI'].apply(lambda x: 'CH' if x > 35 else 'CL')
    return df

if df_soil is None:
    df_soil = create_synthetic_data()

# ==========================================
# 2. Computational Logic
# ==========================================

class RiverbankSoil:
    def __init__(self, dataframe):
        self.df = dataframe.copy()

    def _calculate_base_A(self, row, force_submerged=False):
        pi_val = max(row['PI'], 0.1)
        liquidity_index = (row['Water_Content'] - row['Liquid_Limit_LL']) / pi_val
        base_A = 2.5 if row['Soil_Type'] == 'CH' else 1.5
        geo_factor = (row['PI'] / row['Water_Content'])
        A = base_A * geo_factor
        A = A / (1 + max(liquidity_index, 0))
        if row['is_submerged'] or force_submerged:
            A = A * 0.8
        return A

    def calculate_strength_at_t(self, t_days, scenario="Baseline"):
        results = []
        for _, row in self.df.iterrows():
            force_submerge = True if scenario == "Flood" else False
            A = self._calculate_base_A(row, force_submerged=force_submerge)
            if scenario == "Flood": A = A * 0.85

            S0 = row['Remolded_Su_S0']
            Cap = 0.75 * row['Undisturbed_Su']
            log_term = np.log10(max(t_days, 1))
            Su_t = min(S0 + (A * log_term), Cap)

            driving_stress = row['Depth_m'] * 5.0
            if scenario == "Flood": driving_stress = driving_stress * 1.10

            FoS = Su_t / driving_stress

            results.append({
                'Sample_ID': row['Sample_ID'],
                'Scenario': scenario,
                'Calculated_Su_kPa': round(Su_t, 2),
                'Recovery_Const_A': round(A, 3),
                'Driving_Stress_kPa': round(driving_stress, 2),
                'FoS': round(FoS, 2)
            })
        return pd.DataFrame(results)

    def run_sensitivity_comparison(self, t_days):
        df_base = self.calculate_strength_at_t(t_days, "Baseline")
        df_flood = self.calculate_strength_at_t(t_days, "Flood")
        return pd.concat([df_base, df_flood], ignore_index=True)

    def calculate_strategic_metrics(self, t_current, target_fos):
        df_cur = self.calculate_strength_at_t(t_current, "Baseline")
        failure_rate = (df_cur['FoS'] < target_fos).mean() * 100

        lags = []
        df_dry_cur = self.calculate_strength_at_t(t_current, "Baseline")
        for _, row in self.df.iterrows():
            sid = row['Sample_ID']
            fos_dry_cur = df_dry_cur[df_dry_cur['Sample_ID']==sid]['FoS'].values[0]
            for t_future in range(t_current, 365):
                df_flood_fut = self.calculate_strength_at_t(t_future, "Flood")
                fos_flood_fut = df_flood_fut[df_flood_fut['Sample_ID']==sid]['FoS'].values[0]
                if fos_flood_fut >= fos_dry_cur:
                    lags.append(t_future - t_current)
                    break
            else: lags.append(365)
        avg_lag = np.mean(lags)

        crit = df_cur.merge(self.df[['Sample_ID', 'Soil_Type']], on='Sample_ID')
        crit_group = crit.groupby('Soil_Type')['FoS'].mean().idxmin()
        return failure_rate, round(avg_lag, 1), crit_group

    def calculate_wait_time(self, target_fos, confidence=0.95):
        for t in range(1, 365):
            df = self.calculate_strength_at_t(t, "Baseline")
            safe_count = (df['FoS'] >= target_fos).sum()
            total = len(df)
            if safe_count / total >= confidence:
                return t
        return "Not Achievable"

soil_model = RiverbankSoil(df_soil)

# ==========================================
# 3. Dynamic PDF Report Generator
# ==========================================

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 15)
        self.cell(0, 10, 'Thixo-Metric Technical Report', 0, 1, 'C')
        self.ln(5)
        self.set_font('Arial', 'I', 10)
        self.cell(0, 5, 'Quantitative Geotechnical Stability Analysis', 0, 1, 'C')
        self.ln(10)

    def chapter_title(self, num, label):
        self.set_font('Arial', 'B', 12)
        self.set_fill_color(200, 220, 255)
        self.cell(0, 8, f'{num}. {label}', 0, 1, 'L', 1)
        self.ln(4)

    def chapter_body(self, body):
        self.set_font('Arial', '', 11)
        self.multi_cell(0, 6, body)
        self.ln()

    def add_table(self, headers, data, title=""):
        if title:
            self.set_font('Arial', 'B', 10)
            self.cell(0, 6, title, 0, 1, 'L')

        self.set_font('Arial', 'B', 10)
        effective_width = self.w - self.l_margin - self.r_margin
        col_width = effective_width / len(headers)

        for header in headers:
            self.cell(col_width, 7, header, 1, 0, 'C')
        self.ln()

        self.set_font('Arial', '', 10)
        for row in data:
            for item in row:
                self.cell(col_width, 6, str(item), 1, 0, 'C')
            self.ln()

def generate_dynamic_report(days, fail_rate, hyd_lag, wait_days, target_fos, df_display, dashboard_img_path, critical_soil, depth_warning):
    pdf = PDF()
    pdf.add_page()

    # 1. Dynamic Executive Summary
    pdf.chapter_title(1, "Executive Summary")
    summary = (f"Analysis was performed at t={days} days since disturbance.\n"
               f"The current Reach Failure Rate is {fail_rate:.1f}%.\n"
               f"Submerged samples experience an average Hydraulic Lag of {hyd_lag} days.\n"
               f"Based on a 95% confidence interval, construction must wait until Day {wait_days}.")
    pdf.chapter_body(summary)

    # 2. High-Risk Identification
    pdf.chapter_title(2, "High-Risk Borehole Identification")
    critical_df = df_display.nsmallest(5, 'FoS')
    table_headers = ['Sample ID', 'FoS', 'Status', 'Soil Type']
    table_data = []
    for _, row in critical_df.iterrows():
        table_data.append([row['Sample_ID'], row['FoS'], row['Status'], row['Soil_Type']])
    pdf.add_table(table_headers, table_data)
    pdf.ln(5)
    pdf.chapter_body("The table above lists the 5 most critical samples requiring immediate monitoring.")

    # 3. Soil Type Vulnerability
    pdf.chapter_title(3, "Soil Classification Vulnerability")
    ch_fos = df_display[df_display['Soil_Type']=='CH']['FoS'].mean()
    cl_fos = df_display[df_display['Soil_Type']=='CL']['FoS'].mean()

    vuln_text = ""
    if ch_fos < cl_fos:
        vuln_text = f"CH (High Plasticity) soils are recovering slower than CL soils. This is likely due to higher Liquidity Index values, indicating a higher state of plasticity and lower initial recovery rates."
    else:
        vuln_text = f"CL (Low Plasticity) soils are underperforming. This suggests external factors (such as submergence or specific mineralogy) are inhibiting recovery in this specific reach."
    pdf.chapter_body(vuln_text)

    # 4. Visual Integration & Analysis
    pdf.chapter_title(4, "Visual Dashboard Analysis")
    pdf.chapter_body("The dashboard (below) visualizes the time-dependent recovery and sensitivity to flooding.")
    pdf.ln(2)

    if dashboard_img_path and os.path.exists(dashboard_img_path):
        pdf.image(dashboard_img_path, x=10, y=None, w=180)
    else:
        pdf.chapter_body("[Dashboard Image Embedding Failed]")

    pdf.ln(10)
    pdf.chapter_body("Visual Analysis: The 'Flood' KDE plot shows a distinct leftward shift compared to the 'Baseline' plot. This shift visually quantifies the Hydraulic Lag penalty, indicating that saturated soil requires significantly more time to reach the same safety factor as dry soil.")

    # 5. Strategic Decision & Depth Analysis
    pdf.chapter_title(5, "Strategic Decision Support")
    decision_text = (f"Recommendation: Do not commence construction before Day {wait_days}.\n"
                     f"This ensures that 95% of the borehole reach maintains stability above the target FoS of {target_fos}.")
    pdf.chapter_body(decision_text)

    if depth_warning:
        pdf.set_font('Arial', 'BI', 10)
        pdf.set_text_color(200, 0, 0)
        pdf.multi_cell(0, 6, f"DEPTH WARNING: {depth_warning}")
        pdf.set_text_color(0, 0, 0)

    pdf.add_page()
    pdf.chapter_title(6, "Assumptions & Limitations")
    tech_text = ("- Driving Stress: Calculated as depth * unit weight (1D approximation).\n"
                "- Cap: Recovery is capped at 75% of Undisturbed Su.\n"
                "- Formula: Su(t) = S0 + [A * log10(t)].\n"
                "- Chemical cementation and aging effects are not considered.")
    pdf.chapter_body(tech_text)

    return pdf.output(dest='S').encode('latin-1')

# ==========================================
# 4. Dashboard & Visualization Logic
# ==========================================

style = {'description_width': 'initial'}
status_output = widgets.Output()
output_area = widgets.Output()

# --- Widgets Configuration ---
uploader = widgets.FileUpload(description='Upload Input as CSV file', accept='.csv, .xlsx', multiple=False, button_style='info')
btn_run = widgets.Button(description="Run Analysis", button_style='warning', icon='play')

# Workflow Buttons
btn_gen_report = widgets.Button(description="Generate Technical Report", button_style='info', icon='file-alt')
btn_dl_report = widgets.Button(description="Download Technical Report", button_style='success', icon='download')
btn_dl_report.disabled = True # Disabled initially

# Individual Downloads
btn_dl_csv = widgets.Button(description="Download CSV", button_style='warning', icon='table')
btn_dl_dashboard = widgets.Button(description="Download Dashboard (PNG)", button_style='warning', icon='image')
btn_dl_graphs = widgets.Button(description="Download Individual Graphs", button_style='warning', icon='chart-area')

# Sliders
days_slider = widgets.IntSlider(value=0, min=0, max=120, step=1, description='Days Since Disturbance:', style=style, continuous_update=False)
fos_target = widgets.FloatSlider(value=1.5, min=1.0, max=3.0, step=0.1, description='Target Safety Factor (FoS):', style=style)

global_data_store = {}

def update_dashboard(days, target_fos):
    global current_plot, analysis_run
    analysis_run = True

    # --- Step 1: Show "Analyzing..." ---
    with status_output:
        clear_output(wait=True)
        display(Markdown('<h3 style="color:orange">⏳ Analyzing... Please wait.</h3>'))

    # --- Step 2: Perform Calculations & Plotting ---
    fail_rate, hyd_lag, crit_soil = soil_model.calculate_strategic_metrics(days, target_fos)
    wait_days = soil_model.calculate_wait_time(target_fos)

    # Store for PDF
    global_data_store['fail_rate'] = fail_rate
    global_data_store['hyd_lag'] = hyd_lag
    global_data_store['wait_days'] = wait_days
    global_data_store['crit_soil'] = crit_soil

    df_compare = soil_model.run_sensitivity_comparison(days)
    df_results = soil_model.calculate_strength_at_t(days, "Baseline")
    df_display = df_soil.merge(df_results, on='Sample_ID')
    df_display['Status'] = df_display['FoS'].apply(lambda x: 'SAFE' if x >= target_fos else 'CRITICAL')

    global_data_store['df_compare'] = df_compare
    global_data_store['df_display'] = df_display

    # Depth Warning Logic
    deep_critical = df_display[(df_display['Depth_m'] > 15) & (df_display['Status'] == 'CRITICAL')]
    depth_warning_text = ""
    if not deep_critical.empty:
        depth_warning_text = f"{len(deep_critical)} critical samples detected below 15m. Structural reinforcement is required."
    global_data_store['depth_warning'] = depth_warning_text

    # Render Plot
    current_plot = plt.figure(figsize=(20, 10))
    st = current_plot.suptitle(f"Thixo-Metric: Quantitative Stability Analysis (t={days} days)", fontsize=16, fontweight='bold', y=1.02)

    # Plot 1: Recovery Curves
    ax1 = plt.subplot(2, 3, 1)
    time_range = np.arange(0, 91)
    global_max_strength = 0
    for sid in df_display['Sample_ID']:
        row = df_display[df_display['Sample_ID'] == sid].iloc[0]
        A = row['Recovery_Const_A']; S0 = row['Remolded_Su_S0']; Cap = 0.75 * row['Undisturbed_Su']
        if Cap > global_max_strength: global_max_strength = Cap
        t_safe = np.maximum(time_range, 1)
        su_curve = np.minimum(S0 + (A * np.log10(t_safe)), Cap)
        color = 'green' if row['Status'] == 'SAFE' else 'red'
        ax1.plot(time_range, su_curve, color=color, alpha=0.6)
        ax1.axhline(y=Cap, color='gray', linestyle=':', linewidth=0.5, alpha=0.5)
    ax1.scatter(days, row['Calculated_Su_kPa'], color='black', zorder=10)
    ax1.set_title('Individual Recovery Curves', fontweight='bold')
    ax1.set_xlabel('Time (days)'); ax1.set_ylabel(r'Strength $S_u$ (kPa)')
    ax1.set_xlim(0, 90); ax1.set_ylim(0, np.ceil(global_max_strength/10)*10)

    # Plot 2: Sensitivity
    ax2 = plt.subplot(2, 3, 2)
    sns.kdeplot(data=df_compare, x='FoS', hue='Scenario', fill=True, alpha=0.4, linewidth=2, ax=ax2, palette={'Baseline': 'blue', 'Flood': 'red'})
    ax2.axvline(x=target_fos, color='black', linestyle='--', linewidth=2)
    ax2.set_title('Sensitivity Analysis', fontweight='bold')

    # Plot 3: Spatial Profile
    ax3 = plt.subplot(2, 3, 3)
    sns.scatterplot(data=df_display, x='Depth_m', y='FoS', hue='Status', size='PI', sizes=(50, 200), palette={'SAFE': 'green', 'CRITICAL': 'red'}, ax=ax3)
    ax3.axhline(y=target_fos, color='black', linestyle='--', label='Target FoS')
    ax3.set_title('Spatial Profile: Risk vs. Depth', fontweight='bold')
    ax3.invert_yaxis()

    # Plot 4: Status Bar
    ax4 = plt.subplot(2, 3, 4)
    status_counts = df_display['Status'].value_counts()
    colors_bar = ['green' if x == 'SAFE' else 'red' for x in status_counts.index]
    status_counts.plot(kind='bar', color=colors_bar, alpha=0.7, ax=ax4)
    ax4.set_title('Safety Status Distribution', fontweight='bold')
    for i, v in enumerate(status_counts): ax4.text(i, v + 0.1, str(v), ha='center')

    # Plot 5: Box Spread
    ax5 = plt.subplot(2, 3, 5)
    sns.boxplot(data=df_compare, x='Scenario', y='FoS', hue='Scenario', ax=ax5, palette={'Baseline': 'lightblue', 'Flood': 'salmon'}, legend=False)
    ax5.axhline(y=target_fos, color='black', linestyle='--')
    ax5.set_title('FoS Distribution Spread', fontweight='bold')

    # Plot 6: Wait Time
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')
    wait_text = (f"STRATEGIC DECISION SUPPORT\n\n"
                 f"Current State: {days} days\n"
                 f"Recommended Wait-Time: {wait_days} days\n\n"
                 f"Interpretation: Based on current recovery rates, "
                 f"{'do not start construction' if isinstance(wait_days, int) and wait_days > days else 'it is safe to start'} "
                 f"until day {wait_days}.")
    ax6.text(0.1, 0.5, wait_text, fontsize=12, verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()
    plt.close(current_plot)

    # --- Step 3: Show "Analysis Completed" & Dashboard ---
    with status_output:
        clear_output(wait=True)
        display(Markdown('<h3 style="color:green">✅ Analysis Completed</h3>'))

    with output_area:
        clear_output(wait=True)

        # KPI Cards
        kpi_html = f"""
        <div style="display: flex; justify-content: space-around; background-color: #f0f2f5; padding: 15px; border-radius: 10px; margin-bottom: 20px;">
            <div style="text-align: center;"><h3 style="color: #d9534f;">{fail_rate:.1f}%</h3><small>Failure Rate</small></div>
            <div style="text-align: center;"><h3 style="color: #f0ad4e;">{hyd_lag} Days</h3><small>Avg. Hydraulic Lag</small></div>
            <div style="text-align: center;"><h3 style="color: #5bc0de;">{crit_soil}</h3><small>Critical Profile</small></div>
            <div style="text-align: center;"><h3 style="color: #5cb85c;">{wait_days} Days</h3><small>Wait-Time</small></div>
        </div>
        """
        display(widgets.HTML(kpi_html))
        display(current_plot)

        # Data Preview
        display(Markdown(f"### Raw Data Preview"))
        display(df_display.head(5).style.map(lambda x: 'color: red; font-weight:bold' if x == 'CRITICAL' else 'color: green', subset=['Status']))

# ==========================================
# 5. Handlers
# ==========================================

def handle_upload(change):
    global df_soil, soil_model, current_data_source
    uploaded_file = change.new
    if uploaded_file:
        try:
            filename = list(uploaded_file.keys())[0]
            file_info = uploaded_file[filename]
            content = file_info['content']
            name = filename
            if name.endswith('.csv'): df_new = pd.read_csv(io.BytesIO(content))
            elif name.endswith('.xlsx'):
                try: df_new = pd.read_excel(io.BytesIO(content), engine='openpyxl')
                except ImportError: print("Error: 'openpyxl' missing"); return
            else: print("Error: Unsupported format."); return

            required_cols = ['Sample_ID', 'Depth_m', 'Undisturbed_Su', 'Remolded_Su_S0', 'PI', 'Water_Content', 'Liquid_Limit_LL', 'is_submerged']
            if any(col not in df_new.columns for col in required_cols): print("Upload Failed: Missing columns"); return
            if 'Soil_Type' not in df_new.columns: df_new['Soil_Type'] = df_new['PI'].apply(lambda x: 'CH' if x > 35 else 'CL')

            df_soil = df_new
            soil_model = RiverbankSoil(df_soil)
            current_data_source = name
            display(Markdown(f"<h4 style='color:green'>✅ Uploaded successfully: {name}</h4>"))
            print("Please click 'Run Analysis' to generate dashboard.")
        except Exception as e: print(f"Error: {e}")

def on_run_click(b):
    global analysis_run
    analysis_run = True
    update_dashboard(days_slider.value, fos_target.value)

def on_value_change(change):
    if analysis_run: update_dashboard(days_slider.value, fos_target.value)

def on_gen_report_click(b):
    global pdf_bytes, btn_dl_report

    with status_output:
        clear_output(wait=True)
        display(Markdown('<h3 style="color:blue">⏳ Generating Technical Report... Please wait.</h3>'))

    # 1. Save Dashboard Image locally
    temp_img_path = 'temp_dashboard.png'
    if current_plot:
        current_plot.savefig(temp_img_path, dpi=300, bbox_inches='tight', facecolor='white')

    # 2. Generate PDF Content
    pdf_bytes = generate_dynamic_report(
        days_slider.value,
        global_data_store['fail_rate'],
        global_data_store['hyd_lag'],
        global_data_store['wait_days'],
        fos_target.value,
        global_data_store['df_display'],
        temp_img_path,
        global_data_store['crit_soil'],
        global_data_store['depth_warning']
    )

    # 3. Update UI
    with status_output:
        clear_output(wait=True)
        display(Markdown('<h3 style="color:green">✅ Technical Report Generated Successfully</h3>'))
        display(Markdown('<p>You can now download the report using the button below.</p>'))

    # Enable Download Button
    btn_dl_report.disabled = False

def on_dl_report_click(b):
    if pdf_bytes:
        with open('Thixo-Metric_Report.pdf', 'wb') as f:
            f.write(pdf_bytes)
        files.download('Thixo-Metric_Report.pdf')
    else:
        print("Please generate the report first.")

def on_dl_csv_click(b):
    if analysis_run:
        csv_filename = 'Thixo-Metric_Data.csv'
        global_data_store['df_compare'].to_csv(csv_filename, index=False)
        files.download(csv_filename)
    else:
        print("Please run analysis first.")

def on_dl_dashboard_click(b):
    if current_plot:
        img_filename = 'Thixo-Metric_Dashboard.png'
        current_plot.savefig(img_filename, dpi=300, bbox_inches='tight', facecolor='white')
        files.download(img_filename)
    else:
        print("Please run analysis first.")

def on_dl_graphs_click(b):
    if not analysis_run:
        print("Please run analysis first.")
        return

    df_disp = global_data_store['df_display']
    df_comp = global_data_store['df_compare']
    target = fos_target.value
    days = days_slider.value

    # 1. Recovery Curves
    fig_rec = plt.figure(figsize=(10, 6))
    ax = fig_rec.add_subplot(111)
    time_range = np.arange(0, 91)
    for sid in df_disp['Sample_ID']:
        row = df_disp[df_disp['Sample_ID'] == sid].iloc[0]
        A = row['Recovery_Const_A']; S0 = row['Remolded_Su_S0']; Cap = 0.75 * row['Undisturbed_Su']
        t_safe = np.maximum(time_range, 1)
        su_curve = np.minimum(S0 + (A * np.log10(t_safe)), Cap)
        color = 'green' if row['Status'] == 'SAFE' else 'red'
        ax.plot(time_range, su_curve, color=color, alpha=0.6)
    ax.scatter(days, row['Calculated_Su_kPa'], color='black', zorder=10)
    ax.set_title(f'Individual Recovery Curves (t={days} days)', fontweight='bold')
    ax.set_xlabel('Time (days)'); ax.set_ylabel(r'Strength $S_u$ (kPa)')
    ax.grid(True, linestyle='--')
    files.download('Thixo-Metric_Recovery.png')
    plt.close(fig_rec)

    # 2. Spatial Profile
    fig_spa = plt.figure(figsize=(10, 6))
    ax = fig_spa.add_subplot(111)
    sns.scatterplot(data=df_disp, x='Depth_m', y='FoS', hue='Status', size='PI', sizes=(50, 200), palette={'SAFE': 'green', 'CRITICAL': 'red'}, ax=ax)
    ax.axhline(y=target, color='black', linestyle='--')
    ax.invert_yaxis()
    ax.set_title('Spatial Profile: Risk vs. Depth', fontweight='bold')
    plt.tight_layout()
    files.download('Thixo-Metric_Spatial.png')
    plt.close(fig_spa)

    # 3. Sensitivity Analysis
    fig_sen = plt.figure(figsize=(10, 6))
    ax = fig_sen.add_subplot(111)
    sns.kdeplot(data=df_comp, x='FoS', hue='Scenario', fill=True, alpha=0.4, linewidth=2, ax=ax, palette={'Baseline': 'blue', 'Flood': 'red'})
    ax.axvline(x=target, color='black', linestyle='--', linewidth=2)
    ax.set_title('Sensitivity Analysis: Flood Impact', fontweight='bold')
    plt.savefig('Thixo-Metric_Sensitivity.png', dpi=300, bbox_inches='tight', facecolor='white')
    files.download('Thixo-Metric_Sensitivity.png')
    plt.close(fig_sen)

# Connect Handlers
uploader.observe(handle_upload, names='value')
btn_run.on_click(on_run_click)
days_slider.observe(on_value_change, names='value')
fos_target.observe(on_value_change, names='value')
btn_gen_report.on_click(on_gen_report_click)
btn_dl_report.on_click(on_dl_report_click)
btn_dl_csv.on_click(on_dl_csv_click)
btn_dl_dashboard.on_click(on_dl_dashboard_click)
btn_dl_graphs.on_click(on_dl_graphs_click)

# Layout Construction
dashboard_header = widgets.HTML("<h1>Thixo-Metric v1.0: Final Edition</h1>")

# Row 1: Analysis Controls
row_1 = widgets.HBox([uploader, btn_run])

# Row 2: Report Workflow
row_2 = widgets.VBox([
    widgets.HBox([btn_gen_report, btn_dl_report]),
    widgets.HTML("<small><i>Note: Click 'Generate' first, then 'Download' to get the PDF.</i></small>")
])

# Row 3: Separate Downloads
row_3 = widgets.HBox([btn_dl_csv, btn_dl_dashboard, btn_dl_graphs])

# Sliders
row_sliders = widgets.HBox([days_slider, fos_target])

# Main Layout
dashboard_controls = widgets.VBox([
    row_1,
    status_output,
    row_2,
    row_3,
    widgets.HTML("<hr>"),
    row_sliders,
    output_area
])

display(widgets.VBox([dashboard_header, dashboard_controls]))

<h4 style='color:green'>✅ Uploaded successfully: Bangladesh_River_Boreholes_50.csv</h4>

Please click 'Run Analysis' to generate dashboard.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>